# Split HIV by Weight

In this example notebook, we will discuss how to use DataSAIL to compute split for the QM9 dataset to compute more challenging splits for machine learning models. Therefore, we first import all necessary tools.

In [ ]:
%%capture
import deepchem as dc
import numpy as np
from rdkit import Chem
from rdkit.Chem.Descriptors import ExactMolWt
from datasail.sail import datasail

### Load the Dataset

Load the dataset from deepchem. As the deepchem dataset contains the targets of QM9 as y1 to y12 as well as weights for every task. Therefore, we rename the targets and remove the weights. Finally, we also convert all RDKit molecules to SMILES strings.

In [2]:
dataset = dc.molnet.load_hiv(featurizer=dc.feat.DummyFeaturizer(), splitter=None)[1][0]
df = dataset.to_dataframe()
df.rename(columns=dict([("y", dataset.tasks[0]), ("X", "SMILES")]), inplace=True)
df["ID"] = [f"Comp{i + 1:06d}" for i in range(len(df))]
df = df[["ID", "SMILES"] + dataset.tasks.tolist()]
df

,ID,SMILES,HIV_active
0,Comp000001,CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...,0.0
1,Comp000002,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...,0.0
2,Comp000003,CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21,0.0
3,Comp000004,Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1,0.0
4,Comp000005,O=S(=O)(O)CCS(=O)(=O)O,0.0
...,...,...,...
41122,Comp041123,CCC1CCC2c3c([nH]c4ccc(C)cc34)C3C(=O)N(N(C)C)C(...,0.0
41123,Comp041124,Cc1ccc2[nH]c3c(c2c1)C1CCC(C(C)(C)C)CC1C1C(=O)N...,0.0
41124,Comp041125,Cc1ccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C...,0.0
41125,Comp041126,Cc1cccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)...,0.0


### Run DataSAIL

Use DataSAIL to split the data into an identity-based single cold split (ICSe) and a cluster-based single cold split (CCSe). We define
  - the techniques as list: ICSe abd CCSe
    The e in the end is important to split the e-data.
  - the spits as list. The values will be normalized to ratios.
  - the names as list. Similarly to the list of split sizes, DataSAIL needs names to name the splits.
  - the number of runs. This will determine how many different splits to compute per technique to compute.
  - the solving algorithm for optimizing the final problem formulation.
  - the type of the dataset in the first axis.
  - the data as mapping from IDs to SMILES strings.
  
For an extensive description of the arguments please refer to the according pages of the documentation.

In [3]:
%%capture
# Compute the distance matrix of the weights for each pair of molecule
dist_mat = np.zeros((len(df), len(df)))
weights = [ExactMolWt(Chem.MolFromSmiles(s)) for s in df["SMILES"].values.tolist()]
for i in range(len(weights)):
    for j in range(i + 1, len(weights)):
        dist_mat = abs(weights[i] - weights[j])

# Use this matrix together with a list of names (i.e. the ids of the molecules in order of the matrix) as distance metric in DataSAIL.
e_splits, f_splits, inter_splits = datasail(
    techniques=["ICSe", "CCSe"],
    splits=[7, 2, 1], 
    names=["train","val", "test"], 
    runs=3, 
    solver="MOSEK",
    e_type="M",
    e_data=dict(df[["ID", "SMILES"]].values.tolist()),
    e_dist=(df["ID"].values.tolist(), dist_mat),
)

TypeError: object of type 'NoneType' has no len()

## The output

Finally, we inspect the e_split object as this holds all the assignments of the datapoints to the splits, for each run and each technique. First, the overall architecture is described, lastly we look at the first five assignments of the CCS run 0.

In [4]:
print(type(e_splits))
for key in e_splits.keys():
    print(f"{key} - Type: {type(e_splits[key])} - Length: {len(e_splits[key])}")
    for run in range(len(e_splits[key])):
        print(f"\tRun {run + 1} - Type: {type(e_splits[key][run])} - {len(e_splits[key][run])} assignments")
print("\n" + "\n".join(f"ID: {idx} - Split: {split}" for idx, split in list(e_splits[key][0].items())[:5]))

NameError: name 'e_splits' is not defined